# GIS GZ – Übung 11: Datenanalyse II <span style="color:red">(Musterlösung)</span>

### Grobziele
* Sie können überprüfen, ob ein Phänomen zufällig im Raum verteilt ist oder nicht. 

### Feinziele
* Sie können die Funktionsweise und Aussagen von Moran's I und Average Nearest Neighbor beschreiben und erklären.
* Sie können Moran's I und Average Nearest Neighbor implementieren und Punktdaten auf Autokorrelation hin analysieren.

### Projekt
* Sie arbeiten an Ihrem Projekt und fokussieren auf die Datenanalyse. 

## Einleitung
In der heutigen Übung betrachten wir die Verteilung von Punktdaten. Insbesondere interessiert uns, ob die Datenpunkte zufällig im Raum verteilt sind oder nicht. Wenn uns nur die Lage der Datenpunkte, nicht aber ihre Attributwerte interessiert, können wir dazu `Average Nearest Neighbor` verwenden. Falls wir untersuchen wollen, ob die Lage und die Attributwerte miteinander in einer Verbindung stehen, so verwenden wir dazu `Moran's I`. 

### Dateien
In der heutigen Übung verwenden wir generisch generierte Punkte direkt in Jupyter Notebook, also ohne eine Datei zu importieren. Tipp: Wir haben jeweils die Standardnormalverteilung gewählt, um die Attributwerte zu samplen. Sie dürfen jedoch die Verteilungen anpassen und beobachten, was sich am Resultat ändert. 

### Globale Settings
#### Import-Statements

In [1]:
%matplotlib notebook
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.pyplot as plt
from sklearn.neighbors.kd_tree import KDTree
from ipywidgets import interactive, interact
import ipywidgets as widgets
import scipy.stats as st
import math

#### Wichtige Variablen

In [2]:
n = 80
width = 6
height = 8
std_dev = 10

### Hilfsfunktionen
#### Distanzfunktion
Dies ist eine simple Distanzfunktion, welche die gemessene Distanz in Relation zur maximalen Distanz setzt. Je kleiner die Distanz, desto näher ist das Gewicht bei 1; je weiter die Distanz, desto eher ist es bei 0. 

In [3]:
def compute_distance_weight(point_i, point_j, max_distance):
    current_distance = math.sqrt((point_i[0]-point_j[0])**2 + (point_i[1]-point_j[1])**2)
    ratio = current_distance / max_distance
    result = 1 - ratio
    return result

#### Plotfunktion
Damit werden die Punkte und die berechneten Werte geplottet.

In [4]:
def plot_ann(sampling):

    if sampling == "random":
        xs = np.random.uniform(0, width, n)
        ys = np.random.uniform(0, height, n)
        zs = np.random.normal(0, std_dev, n)  # add a normally distributed column
        pts = np.vstack((xs, ys, zs)).T

    elif sampling == "clustered":
        centers = [(5, 3), (2, 4)]
        cluster_std = [0.3, 0.5]

        pts, c = make_blobs(n_samples=n, cluster_std=cluster_std, centers=centers, n_features=2, random_state=0)
        pts = np.insert(pts, 2, np.random.normal(0, std_dev, n), axis=1)  # add a normally distributed column

    elif sampling == "regular":
        nx = int(np.sqrt(n * 1.9))
        X, Y = np.mgrid[0:width:complex(0, nx), 0:height:complex(0, nx)]
        pts = np.vstack([X.ravel(), Y.ravel()]).T
        pts = np.insert(pts, 2, np.random.normal(0, std_dev, X.size), axis=1)  # add a normally distributed column

        idx = np.random.choice(pts.shape[0], n, replace=False)
        pts = pts[idx, :]
    
    
    # Compute Moran's I
    mi = compute_morans_i(pts)
    if isinstance(mi, float):
        mi = round(mi, 4)
        
    # Compute ANN
    mins = np.min(pts, axis=0)
    maxs = np.max(pts, axis=0)

    N = pts.shape[0]
    A = (maxs[0] - mins[0]) * (maxs[1] - mins[1])

    # Calculate expected distance
    D_e = 0.5 / np.sqrt(N / A)

    # Calculate observed distance
    tree = KDTree(pts, leaf_size=2)
    dist, _ = tree.query(pts, k=2)
    D_o = np.mean(dist, axis=0)[1]

    ANN = D_o / D_e
    
    s = 0.26136 / np.sqrt(N**2 / A)
    z_score = (D_o - D_e) / s
    
    p_value = 1.0 - st.norm.cdf(z_score) 

    fig = plt.figure(figsize=(10, 10))


    cluster_trend = "Clustering (ANN < 1)"
    if ANN > 1:
        cluster_trend = "Dispersion (ANN > 1)"

    # Z: {} P: {}

    plt.title("Moran's I: {}  ||  {}: ANN: {} --> Trend: {}".format(mi, 
                                                                    sampling.capitalize(),
                                                                    round(ANN, 4),
                                                                    cluster_trend,
                                                                    round(z_score, 2),
                                                                    round(p_value, 2)))

    # Data points
    plt.scatter(pts[:, 0],
                pts[:, 1],
                s=12,
                c=pts[:, 2],    # COMMENT/UNCOMMENT THIS LINE TO DISTINGUISH THE Z VALUES BY COLOR
                cmap='Purples'
               )

    plt.xlim(mins[0] - 0.5, maxs[0] + 0.5)
    plt.ylim(mins[1] - 0.5, maxs[1] + 0.5)

    # ax = ...
    text = plt.text(0,0, "", va="bottom", ha="left")

    def onclick(event):
        tx = 'button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
        text.set_text(tx)

    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    
    plt.show()

## Aufgaben
### Moran's I
In der folgenden Funktion haben wir Ihnen die Werte des XYZ-Arrays in drei separate Listen gepackt, was das Iterieren durch die einzelnen Punkte erleichtert. Implementieren Sie nun selbstständig die Funktion, die Ihnen Moran's I berechnet.

In [5]:
def compute_morans_i(pts):
    mins = np.min(pts, axis=0)
    maxs = np.max(pts, axis=0)
    max_distance = math.sqrt((maxs[0]-mins[0])**2 + (maxs[1]-mins[1])**2)
    
    X = pts[:, 0]
    X = X.tolist()
    Y = pts[:, 1]
    Y = Y.tolist()
    Z = pts[:, 2]
    Z = Z.tolist()
    
    n = len(X)
    x_dash = sum(Z) / len(Z)
    
    sum_of_weights = 0
    upper_part = 0
    lower_part = 0
    
    for i in range(n):
        point_i = (X[i], Y[i])
        x_i = Z[i]
        x_i_diff = x_i - x_dash
        lower_part += x_i_diff ** 2
        
        for j in range(n):
            point_j = (X[j], Y[j])
            x_j = Z[j]
            x_j_diff = x_j - x_dash
            
            # Continue, if the points are equal
            if point_i == point_j:
                continue
        
            w_i_j = compute_distance_weight(point_i, point_j, max_distance)
            sum_of_weights += w_i_j
            
            product = w_i_j * x_i_diff * x_j_diff
            upper_part += product
            
    morans_i = n / sum_of_weights * upper_part / lower_part
    return morans_i

### Ausprobieren der interaktive Ausgabefunktion
Aufgrund der Einstellungen oben können Sie hier wählen, ob Sie die Punkteverteilung geclustert, homogen oder zufällig möchten. Beobachten Sie, wie sich die Werte anhand der Punkteverteilung ändern. Wenn Sie Moran's I implementiert haben, so testen Sie auch, ob und was sich ändert, wenn Sie statt der Normalverteilung eine andere Verteilung wählen. 

In [6]:
interact(plot_ann,
         sampling=widgets.Dropdown(
             options=['random', 'clustered', 'regular'],
             value='clustered',
             description='Number:',
             disabled=False,)
         )

interactive(children=(Dropdown(description='Number:', index=1, options=('random', 'clustered', 'regular'), val…

<function __main__.plot_ann(sampling)>